## Include required libararies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits import mplot3d
from datetime import datetime, date
from xgboost import XGBClassifier
from xgboost import plot_importance
import seaborn as sns

## Load the Data

In [2]:
# Path to the csv file
dataPath = "./data/data.csv"

# Load the data
rawDataFrame = pd.read_csv(dataPath)

# Print some data
rawDataFrame.head()

,UNIQUEID,DISBURSED_AMOUNT,ASSET_COST,LTV,BRANCH_ID,SUPPLIER_ID,MANUFACTURER_ID,CURRENT_PINCODE_ID,DATE_OF_BIRTH,EMPLOYMENT_TYPE,...,SEC_SANCTIONED_AMOUNT,SEC_DISBURSED_AMOUNT,PRIMARY_INSTAL_AMT,SEC_INSTAL_AMT,NEW_ACCTS_IN_LAST_SIX_MONTHS,DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS,AVERAGE_ACCT_AGE,CREDIT_HISTORY_LENGTH,NO_OF_INQUIRIES,LOAN_DEFAULT
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-1984,Salaried,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-1985,Self employed,...,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-1985,Self employed,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-1993,Self employed,...,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-1977,Self employed,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1


In [3]:
rawDataFrame.columns

Index(['UNIQUEID', 'DISBURSED_AMOUNT', 'ASSET_COST', 'LTV', 'BRANCH_ID',
       'SUPPLIER_ID', 'MANUFACTURER_ID', 'CURRENT_PINCODE_ID', 'DATE_OF_BIRTH',
       'EMPLOYMENT_TYPE', 'DISBURSAL_DATE', 'STATE_ID', 'EMPLOYEE_CODE_ID',
       'MOBILENO_AVL_FLAG', 'AADHAR_FLAG', 'PAN_FLAG', 'VOTERID_FLAG',
       'DRIVING_FLAG', 'PASSPORT_FLAG', 'PERFORM_CNS_SCORE',
       'PERFORM_CNS_SCORE_DESCRIPTION', 'PRI_NO_OF_ACCTS', 'PRI_ACTIVE_ACCTS',
       'PRI_OVERDUE_ACCTS', 'PRI_CURRENT_BALANCE', 'PRI_SANCTIONED_AMOUNT',
       'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 'SEC_ACTIVE_ACCTS',
       'SEC_OVERDUE_ACCTS', 'SEC_CURRENT_BALANCE', 'SEC_SANCTIONED_AMOUNT',
       'SEC_DISBURSED_AMOUNT', 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT',
       'NEW_ACCTS_IN_LAST_SIX_MONTHS', 'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS',
       'AVERAGE_ACCT_AGE', 'CREDIT_HISTORY_LENGTH', 'NO_OF_INQUIRIES',
       'LOAN_DEFAULT'],
      dtype='object')

We can see that there are 233,154 datapoints in the total dataset.

## Drop Null Values

Now we can simply identify the columns that contain NULL values.

In [4]:
# Identify the features with more than 50% NULL Data
rawDataFrame.isna().sum(axis=0) # We use axis = 0 so the null values are counted column-wise

UNIQUEID                                  0
DISBURSED_AMOUNT                          0
ASSET_COST                                0
LTV                                       0
BRANCH_ID                                 0
SUPPLIER_ID                               0
MANUFACTURER_ID                           0
CURRENT_PINCODE_ID                        0
DATE_OF_BIRTH                             0
EMPLOYMENT_TYPE                        7661
DISBURSAL_DATE                            0
STATE_ID                                  0
EMPLOYEE_CODE_ID                          0
MOBILENO_AVL_FLAG                         0
AADHAR_FLAG                               0
PAN_FLAG                                  0
VOTERID_FLAG                              0
DRIVING_FLAG                              0
PASSPORT_FLAG                             0
PERFORM_CNS_SCORE                         0
PERFORM_CNS_SCORE_DESCRIPTION             0
PRI_NO_OF_ACCTS                           0
PRI_ACTIVE_ACCTS                

We see that the feature `EMPLOYMENT_TYPE` is the only column containing NULL Values. Now there can be 2 options that can be either to drop the entire `EMPLOYMENT_TYPE` feature or just drop the rows that contain the NULL values for this feature. Now due to the large number of total datapoints in the entire dataset (233,154), the amount of NULL values (7,661) is considerable low which is only `3.29%` so it is better to drop the rows instead of droping the whole column as it may contain information that can effect the target variable and losing 3% of the data is not that harmful because we have enough datapoints to generalize on.

In [ ]:
# Drop rows with null values
print(f"Total Number of Datapoints before droping: {len(rawDataFrame)}")
nullDroppedDataframe = rawDataFrame.dropna()
print(f"Total Number of Datapoints after droping: {len(nullDroppedDataframe)}")
print(f"Total Number of Datapoints dropped: {len(rawDataFrame) - len(nullDroppedDataframe)}")

We see that those 7,661 rows are dropped. Great!

## Add a new Feature of Age

Now let's add a new `Age` feature and compute it using the current date.

In [ ]:
# A simple function to get age based on the Date of Birth
def getAge(dateOfBirth):
    
    # Convert string to datetime object and get the date only as `strptime` returns the time as well 
    dateOfBirth = datetime.strptime(dateOfBirth, "%d-%m-%Y").date() # Format Code List can be found at: https://www.programiz.com/python-programming/datetime/strptime
    
    # Get the current date
    today = date.today()
    
    # Return the age by subtracting date of birth from current date
    # We will also consider the dob month and day
    # If the current date's month and day is less than the dob's month and day, we subtract an additional year.
    return today.year - dateOfBirth.year - ((today.month, today.day) < (dateOfBirth.month, dateOfBirth.day))

# Apply the `getAge` method on the entire `DATE_OF_BIRTH` column and create a new feature age    
ageDataframe = nullDroppedDataframe['DATE_OF_BIRTH'].apply(getAge)

# Add the new `AGE` feature in our dataframe next to `DATE_OF_BIRTH`
# Get the Index of `DATE_OF_BIRTH` column in data frame
dateOfBirthColumnIndex = nullDroppedDataframe.columns.get_loc('DATE_OF_BIRTH')

# Add `AGE` column next to `DATE_OF_BIRTH` column
nullDroppedDataframe.insert(dateOfBirthColumnIndex + 1, "AGE", ageDataframe)

# Print the head of Dataframe to see the new feature
nullDroppedDataframe.head()

We see that the `Age` Feature has been added next to `DATE_OF_BIRTH` successfuly.

## 1: Feature Correlation with Target Variable

First step is to see how many features are `categorical` in nature. The features that are `categorical` should be converted to numerical features first before measuring the correlation.

In [ ]:
# Let's find out the names of the columns that are Categorical in nature

# All column names
allColumns = nullDroppedDataframe.columns

# Column names that only contain numeric data
numericColumns = nullDroppedDataframe._get_numeric_data().columns

# Columns that contain categorical data
nonNumericFeatures = list(set(allColumns) - set(numericColumns))

# Let's see the non Numeric Features
nullDroppedDataframe[nonNumericFeatures].head()

In [ ]:
# Let's also find the Unique category values for categorical Variables
uniqueDf = nullDroppedDataframe[nonNumericFeatures].nunique().to_frame().reset_index()
uniqueDf.columns = ['Variable', 'Distinct Count']
uniqueDf

We can observe the from all 6 of these `categorical variables` it only makes sence to use `EMPLOYMENT_TYPE` for our analysis purposes as all other features are simply dates or having too many categories. 

So let's just drop the irrelevant columns and convert Categorical Columns to Numeric columns. For this purpose we can go with Pandas `.astype('category').cat.codes`.

In [ ]:
# Drop the non required features
processedDataFrame = nullDroppedDataframe.drop(['AVERAGE_ACCT_AGE', 'CREDIT_HISTORY_LENGTH', 'DATE_OF_BIRTH', 'DISBURSAL_DATE', 'PERFORM_CNS_SCORE_DESCRIPTION'], axis=1)

# Convert Categorical Features to Numerical Feature
processedDataFrame['EMPLOYMENT_TYPE'] = processedDataFrame['EMPLOYMENT_TYPE'].astype('category').cat.codes
processedDataFrame.head()

So we have got ridden on `Non-Numeric` data from our dataset and we now only have `Numeric` data. Great!

Now let's start the analysis by calculating feature correlations. For this purpose we can use a heatmap to get an idea of the features with each other.

In [ ]:
# Calculate Feature Correlation Matrix
featureCorrelationMatrix = processedDataFrame.corr()

# Plot the `featureCorrelationMatrix` using `seaborn` heatmap
plt.figure(figsize=(22, 10))
heatmap = sns.heatmap(featureCorrelationMatrix, vmin=-1, vmax=1)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=12);

In [ ]:
# Check the correlation against the target Variable
correlationTarget = featureCorrelationMatrix['LOAN_DEFAULT']

# Selecting Features with positive correltaion
featuresOfInterest = correlationTarget[correlationTarget > 0].sort_values()
featuresOfInterest

These are the features that are positively correlated with the target variable, so we will continue with these features.

In [ ]:
# Create a Positively Correlated Dataframe
positivlyCorrelatedDataframe = processedDataFrame[featuresOfInterest.index]
positivlyCorrelatedDataframe.head()

## 2: Demographic Data Impact

For the feature of age we can see that there is a negative correlation. So this feature doesn't have an impact on the target variable.

In [ ]:
processedDataFrame[['AGE', 'LOAN_DEFAULT']].corrwith(processedDataFrame['LOAN_DEFAULT'])

Where as the feature of `BRANCH_ID` has a correlation above the threshold of `0.02`. But we can see below in the `XGBoost Section` that the feature of `BRANCH_ID` has an `F-Score` of only 223 which is below the threshold of 400 that we will chose to proceed with. So my conclusion will be that both of these features don't play a major role in the decision.

In [ ]:
processedDataFrame[['BRANCH_ID', 'LOAN_DEFAULT']].corrwith(processedDataFrame['LOAN_DEFAULT'])

## 3: Features for feature analysis

Feature that are more than 0.02 correlation can be used for feature analysis as they have a higher imapct on the target variable.

In [ ]:
featAnalysisFeatures = correlationTarget[correlationTarget > 0.02].sort_values()
featAnalysisFeatures

We can also use XGBoostClassifier to know the feature importance.

In [ ]:
# Separate Dependent and Independent Varibale
X = positivlyCorrelatedDataframe[featAnalysisFeatures.index].iloc[:, :-1]
y = positivlyCorrelatedDataframe[featAnalysisFeatures.index].iloc[:, -1]

In [ ]:
# We can also use `XGBoostClassifier` to check feature importance

# Train XGBClassifier with our X, y
model = XGBClassifier(use_label_encoder=False)
model.fit(X, y)

# plot feature importance
plot_importance(model)
plt.show()

Let's rely on the feature importance showed to us by `XGBClassifier` and get the features that have f-score more than 400

In [ ]:
# Get features that have `F-Score` more than 400
X = positivlyCorrelatedDataframe[['UNIQUEID', 'CURRENT_PINCODE_ID', 'LTV', 'EMPLOYEE_CODE_ID', 'DISBURSED_AMOUNT', 'SUPPLIER_ID']].iloc[:, :-1]

## 4: Estimator Recommendation

As we have too many features and it is not able to visualize them in 22 dimensions, we can use `PCA(Principle Component Analysis)` for dimentionality reduction. We use PCA with 2 components so we get two features in return. 

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# Ready the PCA
pca = PCA(n_components=2).fit_transform(X, y)

# Get feat1 and feat2
feat1, feat2 = pca[:, 0], pca[:, 1]

In [ ]:
# Plot in 3 Dimensions. 2 Dimensions will be the features and the 3rd dimension will be the target
ax = plt.axes(projection='3d')
ax.figure.set_size_inches(15, 15)
zline = y
xline = feat1
yline = feat2
ax.scatter3D(zline, xline, yline, c=y.values, cmap=matplotlib.colors.ListedColormap(['blue', 'red']), label=['Bla', 'another']);
ax.set_title("3D Plot for 2 PCA Components against the Target Variable", fontsize=18)
ax.set_xlabel("Target", fontsize=12)
ax.set_ylabel("PCA1", fontsize=12)
ax.set_zlabel("PCA2", fontsize=12)
loc = np.arange(0,max(y.values),max(y.values)/float(2))
# ax.set_ticks(loc)
# ax.set_ticklabels(['blue', 'red'])

We can see that there is a perfect gap between the 2 classes. So we can use 

## Supervised Learning
##### 1) Deep Neural Network with a Single output Neuron with Softmax or Sigmoid activation
##### 2) Logistic Regression
##### 3) Naive Bays
##### 4) SVM (Support Vector Machine)
##### 5) Decision Trees 
